In [ ]:
import re
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Chrome
from selenium.webdriver import Firefox

In [ ]:
def get_tweet_text(card):
    plainText=card.find_element('xpath','//div[@data-testid="tweetText"]').text
    return plainText
    
    
def get_tweet_data(card):
    
    """Extract data from tweet card"""
    username = card.find_element('xpath','.//span').text
    try:
        plainText=card.find_element('xpath','.//div[@data-testid="tweetText"]').text
        handle = card.find_element('xpath','.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element('xpath','.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element('xpath','.//div[2]/div[2]/div[1]').text
    responding = card.find_element('xpath','.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element('xpath','.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element('xpath','.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element('xpath','.//div[@data-testid="like"]').text
    
    # get a string of all emojis contained in the tweet
    """Emojis are stored as images... so I convert the filename, which is stored as unicode, into 
    the emoji character."""
    emoji_tags = card.find_elements('xpath','.//img[contains(@src, "emoji")]')
    emoji_list = []
    for tag in emoji_tags:
        filename = tag.get_attribute('src')
        try:
            emoji = chr(int(re.search(r'svg\/([a-z0-9]+)\.svg', filename).group(1), base=16))
        except AttributeError:
            continue
        if emoji:
            emoji_list.append(emoji)
    emojis = ' '.join(emoji_list)
    
    tweet = (username, handle, postdate, text, emojis, reply_cnt, retweet_cnt, like_cnt,plainText)
    return tweet    

In [ ]:
user='zenitsugod69@gmail.com'
my_password='tablet01'
# create instance of web driver
# options = EdgeOptions()
# options.use_chromium = True
service = Service(executable_path='E:\\chromedriver_win32\chromedriver.exe')
driver = Chrome(service=service)



In [ ]:
# navigate to login screen
driver.get("https://www.twitter.com/login")
driver.maximize_window()
sleep(5)
username = driver.find_element('xpath','//input[@name="text"]')
username.send_keys(user)
username.send_keys(Keys.RETURN)
sleep(3)

password = driver.find_element('xpath','//input[@name="password"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)
sleep(3)



In [ ]:
# find search input and search for term
search_term='swachh bharat'
search_input = driver.find_element('xpath','//input[@aria-label="Search query"]')
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)
sleep(1)

# navigate to historical 'latest' tab


In [ ]:
#data = driver.find_elements('xpath','//div[@data-testid="tweetText"]')

data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True
page_cards = driver.find_elements('xpath','//article[@data-testid="tweet"]')
# //print(get_tweet_text(page_cards[0]))
tweet = [get_tweet_data(card) for card in page_cards]
# print(tweet)
while scrolling:
    page_cards = driver.find_elements('xpath','//article[@data-testid="tweet"]')
    for card in page_cards[-100:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
               # print(data)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 10:
                scrolling = False
                break
            else:
                sleep(6) # attempt another scroll
        else:
            last_position = curr_position
            break

In [ ]:
len(data)
with open('swachh_bharat.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Emojis', 'Comments', 'Likes', 'Retweets','PlainText']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

In [ ]:
# for dp in data:
#     print(dp[0]+"  -> "+dp[8])
len(data)


157